In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

een ingebouwde functie binnen keras.preprocessing die een BatchDataSet object aanmaakt met de opgegeven parameters

In [ ]:
train_dataset = image_dataset_from_directory("/content/drive/MyDrive/deeplearning/schilderijen/selected_images/train",
                                             image_size=(256,256),
                                             labels = 'inferred',
                                             batch_size = 8,
                                             label_mode = 'categorical')
validation_dataset = image_dataset_from_directory("/content/drive/MyDrive/deeplearning/schilderijen/selected_images/val",
                                                  image_size=(256,256),
                                                  labels = 'inferred',
                                                  batch_size = 8,
                                                  label_mode = 'categorical')
test_dataset = image_dataset_from_directory("/content/drive/MyDrive/deeplearning/schilderijen/selected_images/test",
                                            image_size=(256,256),
                                            labels = 'inferred',
                                            batch_size = 8,
                                            label_mode = 'categorical')


de convolutional base van VGG16 in een variabele steken

In [15]:
conv_base = keras.applications.VGG16(
 weights="imagenet",
 include_top=False)
conv_base.trainable = False

het model aanmaken met data augmentation, softmax, categorical_crossentropy en RMSprop
eerst wordt de shape van de inputs meegegeven, daarna word de data augmentatie toegepast omdat we met een kleine dataset werken
daarna worden de inputs doorgegeven aan de ingebouwde prepocessing van Xception waardoor we betere resultaten zullen krijgen
nadat de images geprocessed zijn geven we ze door aan de conv_base
daarna passen we er een softmax activatie op toe zodat we een vector krijgen van de kansen dat een image bij een bepaalde klasse hoort

In [19]:
data_augmentation = keras.Sequential(
 [
layers.experimental.preprocessing.RandomFlip("horizontal"),
layers.experimental.preprocessing.RandomRotation(0.1),
layers.experimental.preprocessing.RandomZoom(0.2),
 ]
)
inputs = keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="categorical_crossentropy",
optimizer="RMSprop",
metrics=["categorical_accuracy"])


het model trainen

In [ ]:
callbacks = [
 keras.callbacks.ModelCheckpoint(
filepath="VGG16.h5",
save_best_only=True,
monitor="val_loss")
]
history = model.fit(
 train_dataset,
 epochs=50,
 validation_data=validation_dataset,
 callbacks=callbacks)

een plot maken van de accuracy en loss van de training- en validatiesets

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["categorical_accuracy"]
val_accuracy = history.history["val_categorical_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "r", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "r", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

het model testen op de testdataset

In [ ]:
test_model = keras.models.load_model(
 "VGG16.h5")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")
# het model testen